<a href="https://colab.research.google.com/github/jbshirk/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/Joseph_Shirk_LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Save to repo: DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments
> module1-statistics-probability-and-inference/

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd

In [0]:
issues_exp = (
#class,# Name: 2 (democrat, republican)
'handicapped-infants'#: 2 (y,n)
,'water-project-cost-sharing'#: 2 (y,n)
,'adoption-of-the-budget-resolution'#: 2 (y,n)
,'physician-fee-freeze'#: 2 (y,n)
,'el-salvador-aid'#: 2 (y,n)
,'religious-groups-in-schools'#: 2 (y,n)
,'anti-satellite-test-ban'#: 2 (y,n)
,'aid-to-nicaraguan-contras'#: 2 (y,n)
,'mx-missile'#: 2 (y,n)
,'immigration'#: 2 (y,n)
,'synfuels-corporation-cutback'#: 2 (y,n)
,'education-spending'#: 2 (y,n)
,'superfund-right-to-sue'#: 2 (y,n)
,'crime'#: 2 (y,n)
,'duty-free-exports'#: 2 (y,n)
,'export-administration-act-south-africa'#: 2 (y,n)
)

In [16]:
issues_exp

('handicapped-infants',
 'water-project-cost-sharing',
 'adoption-of-the-budget-resolution',
 'physician-fee-freeze',
 'el-salvador-aid',
 'religious-groups-in-schools',
 'anti-satellite-test-ban',
 'aid-to-nicaraguan-contras',
 'mx-missile',
 'immigration',
 'synfuels-corporation-cutback',
 'education-spending',
 'superfund-right-to-sue',
 'crime',
 'duty-free-exports',
 'export-administration-act-south-africa')

In [20]:
issues = pd.DataFrame({'issue':issues_exp})
issues

,issue
0,handicapped-infants
1,water-project-cost-sharing
2,adoption-of-the-budget-resolution
3,physician-fee-freeze
4,el-salvador-aid
5,religious-groups-in-schools
6,anti-satellite-test-ban
7,aid-to-nicaraguan-contras
8,mx-missile
9,immigration


In [0]:
cols = ('party', '0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15')

In [0]:
csv = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data',
               names=cols   )

In [24]:
csv.shape

(435, 17)

In [25]:
csv.sample(20)

,party,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
391,democrat,y,y,n,n,n,n,n,y,y,n,y,n,n,n,y,n
333,democrat,n,n,y,n,n,n,y,y,y,n,n,n,n,n,y,y
300,republican,n,n,n,y,y,n,y,y,y,y,n,y,y,y,n,y
268,democrat,y,n,y,n,n,n,y,y,y,n,n,n,n,n,y,?
156,republican,y,y,n,y,y,y,y,n,n,n,n,y,y,y,n,n
321,democrat,y,y,y,n,n,n,y,y,y,y,n,n,y,n,n,y
123,republican,y,?,n,y,y,y,n,y,n,n,n,y,y,y,n,y
130,democrat,y,?,y,n,?,?,y,y,y,n,n,n,n,n,y,?
290,democrat,y,n,y,n,?,y,?,y,y,y,n,n,y,y,n,y
263,democrat,y,n,y,n,n,n,y,y,y,n,n,n,n,n,y,?


In [0]:
import numpy as np
def yea_nay(vote):
  if vote == 'y':return 1
  if vote == 'n':return -1
  if vote == '?':return 0
  else: return vote #ignore party

In [0]:
#csv.iloc[:,1:17].apply(lambda col: yea_nay(col))

In [0]:
#csv.apply(lambda col: yea_nay(col) if col.name != 'party' else col)

In [0]:
#csv[['0','1']].apply(yea_nay,axis=1)

In [53]:
csv.columns.size-1

16

In [0]:
csv[csv.columns[-16:]]

In [0]:
csv[csv.columns[-16:]].applymap(lambda x: yea_nay(x))

In [0]:
votes[votes.columns[-16:]] = csv[csv.columns[-16:]].applymap(lambda x: yea_nay(x))

In [61]:
votes.head()

,party,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,republican,-1,1,-1,1,1,1,-1,-1,-1,1,0,1,1,1,-1,1
1,republican,-1,1,-1,1,1,1,-1,-1,-1,-1,-1,1,1,1,-1,0
2,democrat,0,1,1,0,1,1,-1,-1,-1,-1,1,-1,1,1,-1,-1
3,democrat,-1,1,1,-1,0,1,-1,-1,-1,-1,1,-1,1,-1,-1,1
4,democrat,1,1,1,-1,1,1,-1,-1,-1,-1,1,0,1,1,1,1
